### Aprendizado de máquina de classificação binária Projeto de estudo de caso

Como você trabalha com um problema de aprendizado de máquina de modelagem preditiva de ponta a ponta? Nisso
lição você trabalhará com um problema de modelagem preditiva de classificação de estudo de caso em Python
incluindo cada etapa do processo de aprendizado de máquina aplicado. Depois de concluir este projeto, você
saberá:

. Como trabalhar com um problema de modelagem preditiva de classificação de ponta a ponta.
. Como usar transformações de dados para melhorar o desempenho do modelo.
. Como usar o ajuste de algoritmo para melhorar o desempenho do modelo.
. Como usar métodos de conjunto e ajuste de métodos de conjunto para melhorar o desempenho do modelo
mance.

#### Definição do Problema

O foco deste projeto será o conjunto de dados Sonar Mines vs Rocks1. O problema é prever
objetos de metal ou rocha dos dados de retorno do sonar. Cada padrão é um conjunto de 60 números no intervalo
0,0 a 1,0. Cada número representa a energia dentro de uma determinada banda de frequência, integrada
durante um determinado período de tempo. O rótulo associado a cada registro contém a letra R se
o objeto é uma rocha e M se for uma mina (cilindro de metal). Os números nas etiquetas estão em
ordem crescente do ângulo de aspecto, mas eles não codificam o ângulo diretamente.

#### Carregar o conjunto de dados

Vamos começar carregando as bibliotecas necessárias para este projeto.

In [ ]:
# Load libraries
import numpy
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

Você pode baixar o conjunto de dados do site do repositório UCI Machine Learning2 e salvar
no diretório de trabalho local com o nome de arquivo sonar.all-data.csv.

In [ ]:
# Load dataset
url = 'sonar.all-data.csv'
dataset = read_csv(url, header=None)

Você pode ver que não estamos especificando os nomes dos atributos desta vez. Isto é porque
além do atributo de classe (a última coluna), as variáveis não possuem nomes significativos.
Também indicamos que não há informações de cabeçalho, isso é para evitar que o código de carregamento do arquivo leve o
primeiro registro como os nomes das colunas. Agora que carregamos o conjunto de dados, podemos dar uma olhada nele.

#### Analisar dados

Vamos dar uma olhada em nossos dados carregados.

#### Estatísticas descritivas

Começaremos confirmando as dimensões do conjunto de dados, por ex. o número de linhas e
colunas.

In [ ]:
# shape
print(dataset.shape)

Temos 208 instâncias para trabalhar e podemos confirmar que os dados têm 61 atributos, incluindo
o atributo de classe.

Vejamos também os tipos de dados de cada atributo.

In [ ]:
# types
set_option('display.max_rows', 500)
print(dataset.dtypes)

Podemos ver que todos os atributos são numéricos (aveia) e que o valor da classe foi
lido como um objeto.

Vamos agora dar uma olhada nas primeiras 20 linhas dos dados.

In [ ]:
# head
set_option('display.width', 100)
print(dataset.head(20))

Isso não mostra todas as colunas, mas podemos ver que todos os dados têm a mesma escala. Nós
também pode ver que o atributo de classe (60) tem valores de string.

Vamos resumir a distribuição de cada atributo.

In [ ]:
# descriptions, change precision to 3 places
set_option('precision', 3)
print(dataset.describe())

Novamente, como esperamos, os dados têm o mesmo intervalo, mas valores médios curiosamente diferentes.
Pode haver algum benefício em padronizar os dados.

Vamos dar uma olhada rápida na divisão dos valores de classe.

In [ ]:
# class distribution
print(dataset.groupby(60).size())

Podemos ver que as classes estão razoavelmente equilibradas entre M (minas) e R (rochas).

##### Visualizações de dados unimodais

Vejamos as visualizações de atributos individuais. Muitas vezes é útil olhar para os seus dados
usando várias visualizações diferentes para gerar ideias. Vejamos os histogramas de cada
atributo para obter uma noção das distribuições de dados.

In [ ]:
# histograms
dataset.hist(sharex=False, sharey=False, xlabelsize=1, ylabelsize=1)
pyplot.show()

Podemos ver que há muitas distribuições gaussianas e talvez algumas exponenciais.
como distribuições para outros atributos.

Vamos dar uma olhada na mesma perspectiva dos dados usando gráficos de densidade.

In [ ]:
# density
dataset.plot(kind='density', subplots=True, layout=(8,8), sharex=False, legend=False,
fontsize=1)
pyplot.show()

Isso é útil, você pode ver que muitos dos atributos têm uma distribuição distorcida. Um poder
transformar como uma transformação Box-Cox que pode corrigir a distorção nas distribuições pode ser
útil.

É sempre bom olhar para gráficos de caixa e bigodes de atributos numéricos para ter uma ideia de
a propagação de valores.

In [ ]:
# box and whisker plots
dataset.plot(kind='box', subplots=True, layout=(8,8), sharex=False, sharey=False,
fontsize=1)
pyplot.show()

Podemos ver que os atributos têm spreads bastante diferentes. Como as escalas são iguais,
pode sugerir algum benefício em padronizar os dados para modelagem para obter todas as médias alinhadas
acima

##### Visualizações de dados multimodais

Vamos visualizar as correlações entre os atributos.

In [ ]:
# correlation matrix
fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(dataset.corr(), vmin=-1, vmax=1, interpolation='none')
fig.colorbar(cax)
pyplot.show()

Parece que também há alguma estrutura na ordem dos atributos. O vermelho ao redor
a diagonal sugere que os atributos que estão próximos uns dos outros são geralmente mais correlacionados
um com o outro. As manchas azuis também sugerem alguma correlação negativa moderada quanto mais
os atributos estão distantes um do outro na ordenação. Isso faz sentido se a ordem do
Os atributos referem-se ao ângulo dos sensores para o chirp do sonar.

#### Conjunto de dados de validação

É uma boa ideia usar um conjunto de validação. Esta é uma amostra dos dados que mantemos
de volta de nossa análise e modelagem. Nós o usamos logo no final do nosso projeto para con rmar o
precisão do nosso modelo nal. É um teste de fumaça que podemos usar para ver se erramos e
nos dê confiança em nossas estimativas de precisão em dados não vistos. Usaremos 80% do conjunto de dados
para modelagem e reter 20% para validação.

In [ ]:
# Split-out validation dataset
array = dataset.values
X = array[:,0:60].astype(float)
Y = array[:,60]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,
test_size=validation_size, random_state=seed)

#### Avaliar algoritmos: linha de base

Não sabemos quais algoritmos se sairão bem neste conjunto de dados. A intuição sugere com base na distância
algoritmos como k-Nearest Neighbors e Support Vector Machines podem funcionar bem. vamos projetar
nosso arnês de teste. Usaremos validação cruzada de 10 vezes. O conjunto de dados não é muito pequeno e isso é
uma boa configuração de chicote de teste padrão. Avaliaremos algoritmos usando a precisão
métrica. Esta é uma métrica bruta que dará uma ideia rápida de quão correto é um determinado modelo. Mais
útil em problemas de classificação binária como este.

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'accuracy'

Vamos criar uma linha de base de desempenho neste problema e verificar no local uma série de diferentes
algoritmos. Vamos selecionar um conjunto de diferentes algoritmos capazes de trabalhar nesta classificação
problema. Os seis algoritmos selecionados incluem:

. Algoritmos Lineares: Regressão Logística (LR) e Análise Discriminante Linear (LDA).
. Algoritmos Não Lineares: Árvores de Classificação e Regressão (CART), Vetor de Suporte
Machines (SVM), Gaussian Naive Bayes (NB) e k-Nearest Neighbors (KNN).

In [ ]:
# Spot-Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

Todos os algoritmos usam parâmetros de ajuste padrão. Vamos comparar os algoritmos. Vamos
exibir a média e o desvio padrão de precisão para cada algoritmo à medida que o calculamos e
coletar os resultados para uso posterior.

In [ ]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

A execução do exemplo fornece a saída abaixo. Os resultados sugerem que tanto a Logística
A regressão e os k-vizinhos mais próximos podem valer a pena um estudo mais aprofundado.

Estes são apenas valores médios de precisão. É sempre bom olhar para a distribuição de precisão
valores calculados em dobras de validação cruzada. Podemos fazer isso graficamente usando caixa e bigode
parcelas

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

Os resultados mostram uma distribuição apertada para KNN que é encorajadora, sugerindo baixa variância.
Os maus resultados para SVM são surpreendentes.

É possível que a distribuição variada dos atributos esteja afetando a precisão
de algoritmos como o SVM. Na próxima seção, repetiremos essa verificação aleatória com um
cópia do conjunto de dados de treinamento.

#### Avaliar algoritmos: padronizar dados

Suspeitamos que as diferentes distribuições dos dados brutos possam estar afetando negativamente a habilidade
de alguns dos algoritmos. Vamos avaliar os mesmos algoritmos com uma cópia padronizada do
conjunto de dados. É aqui que os dados são transformados de forma que cada atributo tenha um valor médio de zero
e um desvio padrão de um. Também precisamos evitar vazamento de dados quando transformamos o
dados. Uma boa forma de evitar vazamentos é usar pipelines que padronizam os dados e constroem o
modelo para cada dobra no chicote de teste de validação cruzada. Dessa forma, podemos obter uma estimativa justa
de como cada modelo com dados padronizados pode funcionar em dados não vistos.

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',
LogisticRegression())])))
pipelines.append(('ScaledLDA', Pipeline([('Scaler', StandardScaler()),('LDA',
LinearDiscriminantAnalysis())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN',
KNeighborsClassifier())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART',
DecisionTreeClassifier())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB',
GaussianNB())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVC())])))
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

A execução do exemplo fornece os resultados listados abaixo. Podemos ver que KNN ainda está fazendo
bem, ainda melhor do que antes. Também podemos ver que a padronização dos dados elevou
a habilidade de SVM para ser o algoritmo mais preciso testado até agora.

Novamente, devemos traçar a distribuição das pontuações de precisão usando gráficos de caixa e bigode.

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

Os resultados sugerem aprofundar os algoritmos SVM e KNN. É muito provável que
a configuração além do padrão pode render modelos ainda mais precisos.

#### Ajuste de Algoritmo

Nesta seção, investigamos o ajuste dos parâmetros para dois algoritmos que se mostram promissores de
a verificação pontual na seção anterior: KNN e SVM.

##### Ajuste KNN

Podemos começar ajustando o número de vizinhos para KNN. O número padrão de vizinhos
é 7. Abaixo, tentamos todos os valores ímpares de k de 1 a 21, cobrindo o valor padrão de 7. Cada valor de k
é avaliado usando validação cruzada de 10 vezes no conjunto de dados padronizado de treinamento.

In [ ]:
# Tune scaled KNN
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
neighbors = [1,3,5,7,9,11,13,15,17,19,21]
param_grid = dict(n_neighbors=neighbors)
model = KNeighborsClassifier()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Podemos imprimir a configuração que resultou na maior precisão, bem como a precisão
de todos os valores experimentados. Executando o exemplo, vemos os resultados abaixo.

Podemos ver que a configuração ótima é K=1. Isso é interessante, pois o algoritmo
faça previsões usando a instância mais semelhante apenas no conjunto de dados de treinamento.

##### Ajuste SVM

Podemos ajustar dois parâmetros-chave do algoritmo SVM, o valor de C (quanto relaxar o
margin) e o tipo de kernel. O padrão para SVM (a classe SVC) é usar o Radial
Kernel de função de base (RBF) com um valor C definido como 1,0. Como com KNN, vamos realizar uma grade
pesquisa usando validação cruzada de 10 vezes com uma cópia padronizada do conjunto de dados de treinamento. Vamos
tente vários tipos de kernel mais simples e valores C com menos viés e mais viés (menor que e
mais de 1,0 respectivamente).

In [ ]:
# Tune scaled SVM
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

A execução do exemplo imprime a melhor configuração, a precisão, bem como as precisões
para todas as combinações de configuração.

Podemos ver que a configuração mais precisa foi SVM com um kernel RBF e um valor C
de 1,5. A precisão de 86,7470% é aparentemente melhor do que a KNN poderia alcançar.

#### Métodos de conjunto( Ensemble )

Outra maneira de melhorar o desempenho dos algoritmos nesse problema é usando
métodos de conjunto. Nesta seção, avaliaremos quatro métodos diferentes de aprendizado de máquina ensemble
algoritmos, dois métodos boosting e dois bagging:

. Métodos de Boosting: AdaBoost (AB) e Gradient Boosting (GBM).
. Métodos de ensacamento: Random Forests (RF) e Extra Trees (ET).

Usaremos o mesmo equipamento de teste de antes, validação cruzada de 10 vezes. Sem padronização de dados
é usado neste caso porque todos os quatro algoritmos de conjunto são baseados em árvores de decisão que são
menos sensível a distribuições de dados.

In [ ]:
# ensembles
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
ensembles.append(('RF', RandomForestClassifier()))
ensembles.append(('ET', ExtraTreesClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

Podemos ver que ambas as técnicas de aumento fornecem fortes pontuações de precisão na casa dos 80 (%)
com configurações padrão. Podemos plotar a distribuição de pontuações de precisão na cruz
dobras de validação.

In [ ]:
# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

Os resultados sugerem que o GBM pode ser digno de um estudo mais aprofundado, com uma média forte e um spread
que se inclina para altos 90s (%) em precisão.

#### Finalizar modelo

O SVM se mostrou mais promissor como um modelo estável e de baixa complexidade para este problema. Em
Nesta seção, vamos finalizar o modelo treinando-o em todo o conjunto de dados de treinamento e fazer
previsões para o conjunto de dados de validação de espera para confirmar nossas conclusões. Uma parte dos resultados foi
que o SVM tem melhor desempenho quando o conjunto de dados é padronizado para que todos os atributos tenham uma média
valor de zero e um desvio padrão de um. Podemos calcular isso a partir de todo o treinamento
conjunto de dados e aplique a mesma transformação aos atributos de entrada do conjunto de dados de validação.

In [ ]:
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = SVC(C=1.5)
model.fit(rescaledX, Y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

Podemos ver que alcançamos uma precisão de quase 86% no conjunto de dados de validação retido. A
pontuação que se aproxima de nossas expectativas estimadas acima durante o ajuste do SVM.

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

### End.